In [1]:
import pandas as pd
from tqdm import trange, tqdm
import numpy as np
import torch
import sys
from collections import defaultdict
sys.path.append('/mnt/nfs/zhangtl/utils/')
from util import myout
import pickle as pkl
import json

import dgl

## embeds

In [55]:
subreddits = pd.read_csv('../../raw_data/reddit/web-redditEmbeddings-subreddits.csv', header=None)
subreddits

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,spiders,0.158972,0.285813,0.226329,-0.183338,-0.163159,-0.188910,0.422705,0.168602,0.178193,...,-0.025041,0.184655,0.111942,-1.125799,0.223129,-0.387323,-0.234882,-0.252297,-0.054315,0.500009
1,askreddit,-0.499114,0.323983,-0.424809,-0.222705,-0.327201,0.412856,1.040088,0.996697,-1.256240,...,0.072151,0.226658,0.535705,0.039861,-0.027087,-0.239952,0.554510,0.676439,-0.328846,0.941547
2,globaloffensivetrade,-0.023145,-1.199374,1.661484,-1.025296,1.424670,-1.179507,-1.035703,0.329301,-0.567272,...,2.998430,-1.235703,-0.326850,4.381238,-1.965378,-2.369674,-0.615389,2.010601,-0.919314,-2.149625
3,fireteams,2.492506,-2.529917,-0.448484,-3.543441,-0.586122,-0.101517,1.837567,-0.215715,-2.460388,...,-1.875214,0.969291,0.497958,2.941132,2.423561,1.315691,-0.959618,0.151845,-0.852492,-0.467526
4,funny,-0.819370,-0.865261,0.301753,0.018787,0.201029,0.070462,0.903895,1.038460,-0.306950,...,1.364194,0.830233,-0.137822,-2.204667,-0.162008,0.390291,-0.556404,0.322684,0.641634,1.320716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51273,33rd,-0.000076,-0.001643,-0.002896,0.000581,-0.003509,-0.000827,0.002095,0.000123,0.004370,...,-0.001570,-0.002246,0.003491,-0.004310,0.001598,0.005168,-0.001156,0.001575,-0.001938,0.001594
51274,bestofvic2015,-0.000203,0.012698,-0.007575,0.006486,-0.001693,0.004743,-0.002401,0.000121,0.001600,...,0.001392,0.001258,0.002970,-0.014106,-0.002713,-0.008648,0.003660,-0.005755,-0.001790,-0.001313
51275,aberystwyth,-0.001563,0.004733,-0.000082,-0.000982,-0.001511,0.001285,0.002187,0.002967,0.000660,...,0.002213,0.000520,-0.000564,-0.010018,0.002507,-0.000651,0.004169,-0.004672,0.002371,0.001073
51276,mail_forwarding,0.009269,0.024779,-0.017018,-0.007228,0.002351,0.003462,-0.008446,0.004544,0.022505,...,-0.002591,0.003482,0.001042,-0.002796,0.010739,-0.009148,0.000363,-0.003144,0.001945,-0.017272


In [56]:
sub2idx = {sub:idx for idx,sub in enumerate(subreddits[0].unique().tolist())}
sub_embs = subreddits.iloc[:, 1:].values
myout(sub2idx, sub_embs)

sub2idx : len=51278, dict([spiders: 0, askreddit: 1, globaloffensivetrade: 2, fireteams: 3, funny: 4, the_donald: 5, ...])
sub_embs : shape=(51278, 300)
[[ 1.589720e-01  2.858130e-01  2.263290e-01 ... -2.522970e-01
  -5.431500e-02  5.000090e-01]
 [-4.991140e-01  3.239830e-01 -4.248090e-01 ...  6.764390e-01
  -3.288460e-01  9.415470e-01]
 [-2.314500e-02 -1.199374e+00  1.661484e+00 ...  2.010601e+00
  -9.193140e-01 -2.149625e+00]
 ...
 [-1.563000e-03  4.733000e-03 -8.200000e-05 ... -4.672000e-03
   2.371000e-03  1.073000e-03]
 [ 9.269000e-03  2.477900e-02 -1.701800e-02 ... -3.144000e-03
   1.945000e-03 -1.727200e-02]
 [ 4.570000e-03  1.240300e-02 -3.630000e-04 ... -2.030000e-04
   3.747000e-03  3.650000e-04]]


## load

In [57]:
papers = pd.read_csv('../../raw_data/reddit/soc-redditHyperlinks-body.tsv', sep='\t')
# papers.columns = ['source', 'target', 'rating', 'time']
papers = papers.sort_values(by=['TIMESTAMP', 'SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT'])
papers

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES
0,leagueoflegends,teamredditteams,1u4nrps,2013-12-31 16:39:58,1,"345.0,298.0,0.75652173913,0.0173913043478,0.08..."
3,nfl,cfb,1u4sjvs,2013-12-31 17:37:55,1,"1124.0,949.0,0.772241992883,0.0017793594306,0...."
1,theredlion,soccer,1u4qkd,2013-12-31 18:18:37,-1,"101.0,98.0,0.742574257426,0.019801980198,0.049..."
5,dogemarket,dogecoin,1u4w7bs,2013-12-31 18:35:44,1,"1328.0,1110.0,0.768825301205,0.0143072289157,0..."
10,gfycat,india,1u5df2s,2013-12-31 22:27:50,1,"2849.0,2467.0,0.704106704107,0.00737100737101,..."
...,...,...,...,...,...,...
286559,electricskateboards,askreddit,68ipb2s,2017-04-30 16:41:53,1,"1876.0,1567.0,0.78144989339,0.00692963752665,0..."
286539,dndbehindthescreen,dndhomebrew,68i9cxs,2017-04-30 16:43:37,1,"10367.0,8693.0,0.731648500048,0.032796373107,0..."
286560,mgtow,dataisbeautiful,68ipeos,2017-04-30 16:52:59,1,"1129.0,975.0,0.795394154119,0.0150575730735,0...."
286558,mmorpg,blackdesertonline,68ip5os,2017-04-30 16:54:08,1,"1100.0,909.0,0.778181818182,0.00181818181818,0..."


In [58]:
papers['time'] = papers['TIMESTAMP'].apply(lambda u: int(''.join(u.split(' ')[0].split('-'))))
papers = papers.sort_values(by=['time', 'SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT'])
papers

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES,time
5,dogemarket,dogecoin,1u4w7bs,2013-12-31 18:35:44,1,"1328.0,1110.0,0.768825301205,0.0143072289157,0...",20131231
10,gfycat,india,1u5df2s,2013-12-31 22:27:50,1,"2849.0,2467.0,0.704106704107,0.00737100737101,...",20131231
0,leagueoflegends,teamredditteams,1u4nrps,2013-12-31 16:39:58,1,"345.0,298.0,0.75652173913,0.0173913043478,0.08...",20131231
3,nfl,cfb,1u4sjvs,2013-12-31 17:37:55,1,"1124.0,949.0,0.772241992883,0.0017793594306,0....",20131231
8,posthardcore,bestof2013,1u5ccus,2013-12-31 23:16:20,1,"3927.0,3488.0,0.719887955182,0.0229182582124,0...",20131231
...,...,...,...,...,...,...,...
286371,vegancirclejerk,vegan,68ei3os,2017-04-30 01:06:17,1,"99.0,98.0,0.737373737374,0.040404040404,0.0404...",20170430
286515,videoediting,videos,68hrbrs,2017-04-30 13:12:57,1,"525.0,454.0,0.765714285714,0.0152380952381,0.0...",20170430
286553,weedstocks,wallstreetbets,68ihozs,2017-04-30 14:28:25,1,"870.0,748.0,0.766666666667,0.00459770114943,0....",20170430
286462,wmaf,hapas,68gyun,2017-04-30 06:57:04,-1,"1171.0,995.0,0.794192997438,0.00597779675491,0...",20170430


## cut

In [59]:
cut_step = len(papers) / 20
papers['new_time'] = (papers.index.values // cut_step).astype(np.int)
papers = papers.sort_values(by=['new_time', 'SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT'])
papers

/tmp/ipykernel_211970/2595461646.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  papers['new_time'] = (papers.index.values // cut_step).astype(np.int)


,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES,time,new_time
631,100movies365days,movieaweek,1ul33ms,2014-01-06 19:38:09,1,"2294.0,1927.0,0.755013077594,0.0130775937228,0...",20140106,0
1820,100movies365days,movieaweek,1v90lzs,2014-01-14 18:59:17,1,"1553.0,1322.0,0.743077913715,0.0180296200901,0...",20140114,0
13022,18bfriendzonest,adviceanimals,21djp2,2014-03-25 18:49:17,-1,"2436.0,2153.0,0.649014778325,0.00697865353038,...",20140325,0
13044,18bfriendzonest,askwomen,21dqe7s,2014-03-25 21:24:40,1,"1394.0,1271.0,0.766857962697,0.0243902439024,0...",20140325,0
13046,18bfriendzonest,redditgetsdrawn,21dqe7s,2014-03-25 21:24:40,1,"1394.0,1271.0,0.766857962697,0.0243902439024,0...",20140325,0
...,...,...,...,...,...,...,...,...
281201,zootopia,place,654e4xs,2017-04-13 03:33:56,1,"1125.0,995.0,0.759111111111,0.016,0.0186666666...",20170413,19
281200,zootopia,thefinalclean,654e4xs,2017-04-13 03:33:56,1,"1125.0,995.0,0.759111111111,0.016,0.0186666666...",20170413,19
281447,zootopia,whowouldwin,65aieis,2017-04-13 20:35:49,1,"421.0,366.0,0.771971496437,0.0142517814727,0.0...",20170413,19
284554,zotac,nvidia,679cp3s,2017-04-24 09:27:05,1,"328.0,288.0,0.75,0.0213414634146,0.03658536585...",20170424,19


In [60]:
for ii in range(0, 20):
    print(ii, len(np.where(papers['new_time'].to_numpy()==ii)[0]))

0 14329
1 14328
2 14328
3 14328
4 14328
5 14328
6 14328
7 14328
8 14328
9 14328
10 14328
11 14328
12 14328
13 14328
14 14328
15 14328
16 14328
17 14328
18 14328
19 14328


## build graph

In [61]:
def update_idx(idx, dic, cnt, feats, feat_dim, no_emb, ff):
    if idx not in dic:
        dic[idx] = cnt
        cnt += 1
        # feats.append(torch.tensor(ff).to(torch.float32))
        if idx in sub2idx:
            feats.append(torch.tensor(sub_embs[sub2idx[idx]]).to(torch.float32))
        else:
            feats.append(torch.rand(feat_dim).to(torch.float32))
            no_emb += 1
    return dic, cnt, feats, no_emb

In [76]:
start_year, end_year = 0, 20
feat_dim = 300
index = {}

id2nid, cnt, no_emb = {}, 0, 0
lst, feats = [], []

for ii in trange(len(papers)):
    year = int(papers['new_time'].iloc[ii])
    source = str(papers['SOURCE_SUBREDDIT'].iloc[ii])
    target = str(papers['TARGET_SUBREDDIT'].iloc[ii])
    weight = int(papers['LINK_SENTIMENT'].iloc[ii])
    ff = [float(item) for item in papers['PROPERTIES'].iloc[ii].split(',')]
    
    id2nid, cnt, feats, no_emb = update_idx(source, id2nid, cnt, feats, feat_dim, no_emb, ff)
    id2nid, cnt, feats, no_emb = update_idx(target, id2nid, cnt, feats, feat_dim, no_emb, ff)
    
    lst.append((id2nid[source], id2nid[target], weight, year))

feat = torch.stack(feats)
src = torch.tensor([item[0] for item in lst])
tgt = torch.tensor([item[1] for item in lst])
rel = torch.tensor([item[2] for item in lst])
tsp = torch.tensor([item[3] for item in lst])

myout(feat, src, tgt, rel, id2nid, id2nid, no_emb)

100%|██████████| 286561/286561 [00:22<00:00, 12961.44it/s]


feat : shape=torch.Size([35776, 300])
tensor([[ 0.0436, -0.1706, -0.0530,  ...,  0.0280,  0.2875, -0.1186],
        [-0.0219, -0.0675,  0.0500,  ..., -0.0290,  0.0323, -0.0093],
        [-0.0133, -0.0827,  0.0739,  ...,  0.0066,  0.0371,  0.0153],
        ...,
        [ 0.7120,  0.8360,  0.4623,  ...,  0.6769,  0.8641,  0.6008],
        [ 0.9072,  0.2796,  0.0980,  ...,  0.4879,  0.0467,  0.2158],
        [ 0.7269,  0.9393,  0.1259,  ...,  0.6472,  0.3138,  0.4427]])
src : shape=torch.Size([286561]), tensor([    0,     0,     2,  ..., 19419, 35775,  7327])
tgt : shape=torch.Size([286561]), tensor([   1,    1,    3,  ...,  142, 1240, 6169])
rel : shape=torch.Size([286561]), tensor([ 1,  1, -1,  ...,  1,  1,  1])
id2nid : len=35776, dict(ovies365days: 0, movieaweek: 1, 18bfriendzonest: 2, adviceanimals: 3, askwomen: 4, redditgetsdrawn: 5, ...])
id2nid : len=35776, dict(ovies365days: 0, movieaweek: 1, 18bfriendzonest: 2, adviceanimals: 3, askwomen: 4, redditgetsdrawn: 5, ...])
start_year 

In [65]:
graph = dgl.graph((src, tgt), num_nodes=len(feat))
graph.ndata['feat'] = feat

nid2id = {vv: kk for kk, vv in id2nid.items()}
graph.ndata['raw_nid'] = torch.arange(len(feat))

graph.edata['rel'] = rel
graph.edata['ts'] = tsp
graph

Graph(num_nodes=35776, num_edges=286561,
      ndata_schemes={'feat': Scheme(shape=(300,), dtype=torch.float32), 'raw_nid': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'rel': Scheme(shape=(), dtype=torch.int64), 'ts': Scheme(shape=(), dtype=torch.int64)})

In [66]:
dataset = 'rdb'
dgl.save_graphs(f'../data/{dataset}/graph.bin', [graph])

In [67]:
json.dump(id2nid, open(f'../data/{dataset}/id2nid.json', 'w'))

## gen cites

In [69]:
start_year, end_year = 0, 20
cites = {}
for year in range(start_year, end_year):
    cites[year] = defaultdict(int)

for ii in trange(len(papers)):
    year = int(papers['new_time'].iloc[ii])
    target = str(papers['TARGET_SUBREDDIT'].iloc[ii])
    cites[year][target] += 1
myout(cites[5])

100%|██████████| 286561/286561 [00:03<00:00, 79300.89it/s]

 : len=3486, dict([keto: 10, writingprompts: 123, askreddit: 346, soccer: 63, runescape: 4, fantheories: 8, ...])


In [70]:
tsp = graph.edata['ts']
ts_vals, ts_cuts = np.unique(tsp.numpy(), return_index=True)
ts_cuts = list(ts_cuts) + [len(tsp.numpy())]

num_ts = len(ts_vals)
ts_infos = np.stack([ts_vals, ts_cuts[0:num_ts], ts_cuts[1:num_ts+1]]).transpose()
myout(ts_cuts, ts_vals, ts_infos)

ts_cuts : len=21, list([0, 14329, 28657, ..., 257905, 272233, 286561])
ts_vals : shape=(20,), [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
ts_infos : shape=(20, 3)
[[     0      0  14329]
 [     1  14329  28657]
 [     2  28657  42985]
 [     3  42985  57313]
 [     4  57313  71641]
 [     5  71641  85969]
 [     6  85969 100297]
 [     7 100297 114625]
 [     8 114625 128953]
 [     9 128953 143281]
 [    10 143281 157609]
 [    11 157609 171937]
 [    12 171937 186265]
 [    13 186265 200593]
 [    14 200593 214921]
 [    15 214921 229249]
 [    16 229249 243577]
 [    17 243577 257905]
 [    18 257905 272233]
 [    19 272233 286561]]


In [71]:
labels = {}
nid2id = {v:k for k,v in id2nid.items()}
for year in range(start_year, end_year):
    left, right = ts_infos[np.where(ts_infos[:, 0]==year)[0][0], 1:]
    nids = graph.edges()[0][left:right].unique().tolist()
    ids = [nid2id[nid] for nid in nids]
    
    pdf = pd.DataFrame({'id': ids, 'nid': nids})
    tbar = trange(year+1, end_year, desc=str(year))
    for yy in tbar:
        cdf = pd.DataFrame({'id': list(cites[yy].keys()), str(yy): list(cites[yy].values())})
        cdf[str(yy)] = cdf[str(yy)].astype('float32')
        
        pdf = pd.merge(pdf, cdf, how='left', on='id')
        tbar.set_postfix(year=year, pdf=len(pdf))
    pdf.fillna(0, inplace=True)
    labels[year] = pdf

18: 100%|██████████| 1/1 [00:00<00:00, 138.39it/s, pdf=4289, year=18]
19: 0it [00:00, ?it/s]


In [72]:
labels[5]

,id,nid,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,askwomen,4,6.0,11.0,11.0,8.0,11.0,4.0,14.0,5.0,11.0,6.0,6.0,14.0,10.0,5.0
1,redditgetsdrawn,5,1.0,3.0,1.0,6.0,2.0,2.0,3.0,3.0,7.0,5.0,4.0,6.0,2.0,2.0
2,askreddit,9,425.0,364.0,338.0,341.0,389.0,287.0,292.0,309.0,349.0,355.0,294.0,304.0,248.0,261.0
3,dogecoin,11,20.0,5.0,59.0,3.0,6.0,3.0,14.0,2.0,2.0,3.0,4.0,4.0,2.0,1.0
4,2007scape,12,8.0,3.0,9.0,4.0,5.0,13.0,14.0,10.0,17.0,11.0,6.0,11.0,13.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4324,zengmfootball,15363,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4325,ziincworks,15365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4326,zmarter,15366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4327,zombicide,15368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
pkl.dump(labels, open(f'../data/{dataset}/labels.pkl', 'wb'))

In [74]:
def cumulative_log(df):
    colsn = list(df.columns)
    for i in range(3, len(colsn)):
        df[colsn[i]] += df[colsn[i-1]]
    df.iloc[:, 2:] = np.log(df.iloc[:, 2:] + 1)
    return df

labels_cum_log = {}
for year in range(start_year, end_year-1):
    labels_cum_log[year] = cumulative_log(labels[year])
labels_cum_log[end_year-2] = labels[end_year-2]
print(len(labels_cum_log))
labels_cum_log[5]

19


,id,nid,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,askwomen,4,1.945910,2.890372,3.367296,3.610918,3.871201,3.951244,4.189655,4.262680,4.406719,4.477337,4.543295,4.682131,4.770685,4.812184
1,redditgetsdrawn,5,0.693147,1.609438,1.791759,2.484907,2.639057,2.772589,2.944439,3.091043,3.367296,3.526361,3.637586,3.784190,3.828641,3.871201
2,askreddit,9,6.054440,6.672033,7.028202,7.292337,7.527256,7.670895,7.798523,7.917901,8.037543,8.146130,8.227910,8.305978,8.365439,8.424419
3,dogecoin,11,3.044523,3.258096,4.442651,4.477337,4.543295,4.574711,4.709530,4.727388,4.744932,4.770685,4.804021,4.836282,4.852030,4.859812
4,2007scape,12,2.197225,2.484907,3.044523,3.218876,3.401197,3.761200,4.043051,4.204693,4.430817,4.553877,4.615120,4.718499,4.828314,4.912655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4324,zengmfootball,15363,0.000000,0.000000,0.693147,1.098612,1.098612,1.098612,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294,1.386294
4325,ziincworks,15365,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4326,zmarter,15366,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4327,zombicide,15368,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [75]:
pkl.dump(labels_cum_log, open(f'../data/{dataset}/labels_cum_log.pkl', 'wb'))